In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.subplots as sub
import dash
from dash import dcc, html, Output, Input, State
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix, top_k_accuracy_score
from sklearn.model_selection import StratifiedKFold
import optuna
from optuna.trial import TrialState
#from optuna.integration import PyTorchIgnitePruningHandler

from functools import partial
import random
import os
import itertools
from PIL import Image
from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torchvision
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset
from torchvision.models import vgg16_bn, resnet50, resnet18, efficientnet_b0, densenet121, ResNet50_Weights, ResNet18_Weights, VGG16_BN_Weights, DenseNet121_Weights, EfficientNet_B0_Weights
from torchvision.utils import make_grid, draw_bounding_boxes, draw_segmentation_masks, draw_keypoints
from torchvision import datasets
from torchvision.transforms import ToTensor, v2, ToPILImage
from torchvision.io import decode_image

# from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
# from ignite.metrics import Accuracy, Loss, RunningAverage, ConfusionMatrix
# from ignite.handlers import ModelCheckpoint, EarlyStopping

from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.amp import GradScaler, autocast

from transformers import ViTForImageClassification, ViTImageProcessor, AutoModelForImageClassification, AutoImageProcessor, Trainer, TrainingArguments
from huggingface_hub import snapshot_download, hf_hub_download
import socket
import json
import sys
import io
import base64

In [5]:
# Add project_root/ to sys.path
#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))) for python executable
# Determine project_root as the parent of the current working directory
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Add project_root to sys.path
sys.path.insert(0, project_root)
from src.data_utils import FGVCAircraftDataset, get_datasets, get_loaders, get_raw
from src import aircraft_utils, models, hyperopt

In [3]:
get_raw()

WindowsPath('C:/Users/chihp/Capstone/FGVCAircraft/fgvc-aircraft-2013b/data')

In [15]:
study = optuna.create_study(study_name="Finest_FGVCAircraft_Variant", direction='maximize', 
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5, interval_steps=3))
study.optimize(hyperopt.objective, timeout=None, n_jobs=1, n_trials=30, gc_after_trial=False, show_progress_bar=True)

print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Visualize optimization results
optuna.visualization.plot_optimization_history(study, target_name='Accuracy', error_bar=True).show()
optuna.visualization.plot_param_importances(study).show() # params=["x", "y"]

[I 2025-08-06 01:15:04,740] A new study created in memory with name: Finest_FGVCAircraft_Variant


  0%|          | 0/30 [00:00<?, ?it/s]

[W 2025-08-06 01:15:04,789] Trial 0 failed with parameters: {'lr': 4.584073928311105e-05, 'batch_size': 16, 'optimizer': 'Adam', 'weight_decay': 0.002235711364847211, 'dropout_rate': 0.38641084583753504, 'backbone': 'ResNet50_CAP', 'scheduler': 'CosineAnnealingLR', 'criterion': 'CrossEntropy'} because of the following error: NameError("name 'CAPResNet' is not defined").
Traceback (most recent call last):
  File "c:\Users\chihp\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\chihp\UMich\SIADS\GitHub\CV_aircraft_classifier_capstone_project\src\hyperopt.py", line 93, in objective
    # Model
            
  File "c:\Users\chihp\UMich\SIADS\GitHub\CV_aircraft_classifier_capstone_project\src\hyperopt.py", line 71, in get_model
    if backbone_name == "ResNet50_CAP":
               ^^^^^^^^^
NameError: name 'CAPResNet' is not defined
[W 2025-08-06 01:15:04,794] Trial 0 failed w

NameError: name 'CAPResNet' is not defined

In [ ]:

# # Plot with custom labels
# plt.plot(range(len([t.value for t in study.trials if t.value is not None])), [t.value for t in study.trials if t.value is not None ], marker='o')
# plt.xlabel("Trial")
# plt.ylabel("Validation Accuracy")
# plt.title("Optimization History")
# plt.grid(True)

In [ ]:
# optuna.visualization.plot_parallel_coordinate(study, target_name='Accuracy').show()
# optuna.visualization.plot_slice(study, target_name='Accuracy').show()

In [ ]:
# optuna.visualization.plot_contour(study, target_name = 'Accuracy').show()
# optuna.visualization.plot_edf(study, target_name = 'Accuracy').show()

In [ ]:
# optuna.visualization.plot_intermediate_values(study).show()
# optuna.visualization.plot_rank(study, target_name = 'Accuracy').show()

In [ ]:
# optuna.visualization.plot_timeline(study).show()